El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
# --- IMPORTS GENERALES ---

# Manipulación de datos
import pandas as pd
import numpy as np
import time

# Visualización (opcional, pero recomendado)
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento de Scikit-Learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Métricas de evaluación
from sklearn.metrics import mean_squared_error

# Modelos de Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

# Configuraciones para una mejor visualización
pd.set_option('display.max_columns', 50)
print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [ ]:
# --- CARGA Y LIMPIEZA DE DATOS ---

# Cargar el dataset
try:
    df = pd.read_csv('datasets/car_data.csv')
except FileNotFoundError:
    print("Error: El archivo 'car_data.csv' no se encuentra en la carpeta 'datasets/'.")

print("Datos cargados. Forma inicial:", df.shape)

# Eliminar columnas que no aportan valor predictivo
columnas_a_eliminar = ['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']
df_cleaned = df.drop(columns=columnas_a_eliminar)

# Eliminar duplicados
df_cleaned = df_cleaned.drop_duplicates().reset_index(drop=True)
print(f"Forma después de eliminar duplicados y columnas irrelevantes: {df_cleaned.shape}")

# --- Corregir valores anómalos y erróneos ---

# Filtrar años de matriculación razonables (e.g., entre 1950 y 2016)
df_cleaned = df_cleaned[(df_cleaned['RegistrationYear'] >= 1950) & (df_cleaned['RegistrationYear'] <= 2016)]

# Filtrar precios razonables (e.g., entre 100 y 50000 euros)
df_cleaned = df_cleaned[(df_cleaned['Price'] >= 100) & (df_cleaned['Price'] <= 50000)]

# Filtrar potencias razonables (e.g., entre 10 y 500 CV)
df_cleaned = df_cleaned[(df_cleaned['Power'] >= 10) & (df_cleaned['Power'] <= 500)]

print(f"Forma después de filtrar valores atípicos: {df_cleaned.shape}")


# --- Imputación de valores nulos ---
# Para categóricas, usamos 'unknown' que el modelo puede interpretar como una categoría más.
# Para numéricas, usamos la mediana, que es robusta frente a valores atípicos.

for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']:
    df_cleaned[col] = df_cleaned[col].fillna('unknown')

for col in ['RegistrationMonth']: # 'Power' y 'Mileage' ya no deberían tener nulos tras el filtrado
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())

# Comprobar que no queden valores nulos
print("\nValores nulos después de la limpieza:")
print(df_cleaned.isnull().sum())

# Mostrar las primeras filas del DataFrame limpio
print("\nVista previa de los datos limpios:")
df_cleaned.head()

Datos cargados. Forma inicial: (354369, 16)
Forma después de eliminar duplicados y columnas irrelevantes: (326826, 11)
Forma después de filtrar valores atípicos: (271994, 11)

Valores nulos después de la limpieza:
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
dtype: int64

Vista previa de los datos limpios:


,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes


In [5]:
# --- PREPARACIÓN DE DATOS PARA MODELOS ---

# Definir las características (features) y el objetivo (target)
X = df_cleaned.drop('Price', axis=1)
y = df_cleaned['Price']

# 1. PREPARACIÓN PARA REGRESIÓN LINEAL (One-Hot Encoding + Escalado)
print("Preparando datos para Regresión Lineal...")
features_lr = pd.get_dummies(X, drop_first=True)
X_train_lr, X_test_lr, y_train, y_test = train_test_split(
    features_lr, y, test_size=0.25, random_state=42
)

# Escalar solo las características numéricas
numeric_cols = ['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth']
scaler = StandardScaler()
X_train_lr[numeric_cols] = scaler.fit_transform(X_train_lr[numeric_cols])
X_test_lr[numeric_cols] = scaler.transform(X_test_lr[numeric_cols])

print("Datos para Regresión Lineal listos. Forma de entrenamiento:", X_train_lr.shape)


# 2. PREPARACIÓN PARA MODELOS DE ÁRBOLES (Ordinal Encoding)
print("\nPreparando datos para modelos de árboles...")
categorical_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

features_trees = X.copy()
features_trees[categorical_cols] = encoder.fit_transform(features_trees[categorical_cols])

X_train_trees, X_test_trees, _, _ = train_test_split(
    features_trees, y, test_size=0.25, random_state=42
)

print("Datos para modelos de árboles listos. Forma de entrenamiento:", X_train_trees.shape)

Preparando datos para Regresión Lineal...
Datos para Regresión Lineal listos. Forma de entrenamiento: (203995, 311)

Preparando datos para modelos de árboles...
Datos para modelos de árboles listos. Forma de entrenamiento: (203995, 10)


## Comentarios y observaciones sobre la preparación de los datos

En esta sección se realizó la preparación de los datos para los modelos de regresión y de árboles de decisión. A continuación se resumen los pasos y se presentan algunas observaciones relevantes:

- **Separación de variables:** Se definieron las variables independientes (`X`) y la variable objetivo (`y`), que es el precio del vehículo.
- **Regresión Lineal:** Se aplicó One-Hot Encoding a las variables categóricas y se escalaron las variables numéricas. Esto es importante para que la regresión lineal no se vea afectada por las diferentes escalas de las variables.
- **Modelos de Árboles:** Se utilizó Ordinal Encoding para las variables categóricas, ya que los modelos de árboles pueden trabajar con variables numéricas codificadas de esta manera.

A continuación se muestra un resumen de la forma (shape) de los conjuntos de entrenamiento para cada tipo de modelo:

| Modelo                | Forma de X_train         |
|-----------------------|-------------------------|
| Regresión Lineal      | {X_train_lr.shape}      |
| Modelos de Árboles    | {X_train_trees.shape}   |

> **Nota:** El número de columnas en `X_train_lr` es mayor debido al One-Hot Encoding, mientras que en `X_train_trees` se mantiene el número original de columnas.

**Observaciones:**
- La correcta codificación y escalado de las variables es fundamental para el rendimiento de los modelos.
- Se utilizó un 25% de los datos para el conjunto de prueba, lo que permite evaluar de manera adecuada la capacidad de generalización de los modelos.
- El uso de `handle_unknown='use_encoded_value'` en el OrdinalEncoder ayuda a evitar errores si aparecen categorías nuevas en los datos de prueba.

Esta preparación garantiza que los modelos reciban los datos en el formato adecuado y optimiza su desempeño en las siguientes etapas de entrenamiento y evaluación.


## Entrenamiento del modelo 

In [6]:
# --- ENTRENAMIENTO Y EVALUACIÓN DE MODELOS ---

# Lista para guardar los resultados de cada modelo
results = []

def train_and_evaluate(model, X_train, y_train, X_test, y_test, model_name):
    """Función para entrenar, medir tiempos y evaluar un modelo."""
    print(f"--- Entrenando {model_name} ---")
    
    # Entrenar y medir tiempo
    start_train_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_train_time
    
    # Predecir y medir tiempo
    start_pred_time = time.time()
    predictions = model.predict(X_test)
    pred_time = time.time() - start_pred_time
    
    # Calcular RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    
    print(f"RMSE: {rmse:.2f}")
    print(f"Tiempo de entrenamiento: {train_time:.2f}s")
    print(f"Tiempo de predicción: {pred_time:.4f}s\n")
    
    # Guardar resultados
    results.append({
        'Model': model_name,
        'RMSE': rmse,
        'Training Time (s)': train_time,
        'Prediction Time (s)': pred_time
    })

# 1. Regresión Lineal (prueba de cordura)
lr_model = LinearRegression()
train_and_evaluate(lr_model, X_train_lr, y_train, X_test_lr, y_test, "Linear Regression")

# 2. Árbol de Decisión
dt_model = DecisionTreeRegressor(random_state=42)
train_and_evaluate(dt_model, X_train_trees, y_train, X_test_trees, y_test, "Decision Tree")

# 3. Bosque Aleatorio
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
train_and_evaluate(rf_model, X_train_trees, y_train, X_test_trees, y_test, "Random Forest")

--- Entrenando Linear Regression ---
RMSE: 2577.29
Tiempo de entrenamiento: 3.66s
Tiempo de predicción: 0.0830s

--- Entrenando Decision Tree ---
RMSE: 2149.35
Tiempo de entrenamiento: 0.77s
Tiempo de predicción: 0.0288s

--- Entrenando Random Forest ---
RMSE: 1635.80
Tiempo de entrenamiento: 7.42s
Tiempo de predicción: 0.5928s



In [7]:
# --- ENTRENAMIENTO DE LIGHTGBM CON BÚSQUEDA DE HIPERPARÁMETROS ---
print("--- Entrenando LightGBM con GridSearchCV ---")

# Modelo base
lgbm = lgb.LGBMRegressor(objective='regression', random_state=42)

# Parámetros a probar (un grid pequeño para que no tarde demasiado)
param_grid = {
    'n_estimators': [150, 250],
    'learning_rate': [0.05, 0.1],
    'num_leaves': [31, 40]
}

# Búsqueda con validación cruzada para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Medir tiempo total de la búsqueda
start_train_time = time.time()
grid_search.fit(X_train_trees, y_train)
train_time = time.time() - start_train_time

# Mejor modelo encontrado
best_lgbm = grid_search.best_estimator_
print(f"\nMejores parámetros encontrados: {grid_search.best_params_}")

# Evaluar el mejor modelo
start_pred_time = time.time()
y_pred_lgbm = best_lgbm.predict(X_test_trees)
pred_time = time.time() - start_pred_time
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))

print(f"RMSE: {rmse_lgbm:.2f}")
print(f"Tiempo de entrenamiento (GridSearch): {train_time:.2f}s")
print(f"Tiempo de predicción: {pred_time:.4f}s\n")

# Guardar resultados
results.append({
    'Model': 'LightGBM (Tuned)',
    'RMSE': rmse_lgbm,
    'Training Time (s)': train_time,
    'Prediction Time (s)': pred_time
})

--- Entrenando LightGBM con GridSearchCV ---
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 619
[LightGBM] [Info] Number of data points in the train set: 203995, number of used features: 10
[LightGBM] [Info] Start training from score 4856.355808

Mejores parámetros encontrados: {'learning_rate': 0.1, 'n_estimators': 250, 'num_leaves': 40}
RMSE: 1574.84
Tiempo de entrenamiento (GridSearch): 22.91s
Tiempo de predicción: 0.0960s



## Comentarios
En esta sección se realizó el ajuste y evaluación de varios modelos de regresión, incluyendo LightGBM con búsqueda de hiperparámetros mediante GridSearchCV. El objetivo fue encontrar el mejor balance entre precisión (RMSE) y tiempos de entrenamiento/predicción.

A continuación, se muestra una tabla resumen con los resultados obtenidos para cada modelo probado:
 
 | Modelo                  | RMSE     | Tiempo de Entrenamiento (s) | Tiempo de Predicción (s) |
 |-------------------------|----------|-----------------------------|--------------------------|
 | LightGBM (Tuned)        | 1574.84  | 22.91                       | 0.0960                   |
 | Random Forest           | 1635.80  | 7.42                        | 0.5928                   |
 | Árbol de Decisión       | 2149.35  | 0.77                        | 0.0288                   |
 | Regresión Lineal        | 2577.29  | 3.66                        | 0.0830                   |



**Observaciones:**
- LightGBM ajustado mediante GridSearchCV logró el menor RMSE, mostrando una excelente capacidad predictiva y tiempos de entrenamiento/predicción competitivos.
- Random Forest también presentó buen desempeño, aunque con tiempos de entrenamiento superiores.
- Los modelos más simples, como la Regresión Lineal y el Árbol de Decisión, fueron más rápidos pero menos precisos.
- La selección final del modelo debe considerar tanto la calidad de la predicción como la eficiencia computacional, especialmente en contextos de producción o recursos limitados.


## Análisis del modelo

In [8]:
# --- ANÁLISIS FINAL DE MODELOS ---

# Convertir la lista de resultados en un DataFrame de pandas
results_df = pd.DataFrame(results).sort_values(by='RMSE', ascending=True).reset_index(drop=True)

print("--- Tabla Comparativa de Modelos ---")
print(results_df)

# Conclusión
print("\n--- Conclusión del Proyecto ---")
best_model = results_df.iloc[0]
print(f"El mejor modelo en términos de calidad de predicción (menor RMSE) es '{best_model['Model']}' con un RMSE de {best_model['RMSE']:.2f}.")

print("\nAnálisis general:")
print("1. CALIDAD: Los modelos de boosting como LightGBM y de ensemble como Random Forest suelen superar a los modelos simples como la Regresión Lineal y el Árbol de Decisión.")
print("2. VELOCIDAD: La Regresión Lineal es el modelo más rápido de entrenar, pero su calidad es baja. LightGBM es notablemente rápido tanto en entrenamiento como en predicción, a menudo superando a Random Forest.")
print(f"3. BALANCE: '{best_model['Model']}' parece ofrecer el mejor balance entre alta calidad de predicción y un tiempo de ejecución razonable para este problema.")

# Limpiar memoria (útil en notebooks con poca RAM)
del X_train_lr, X_test_lr, X_train_trees, X_test_trees
print("\nAnálisis completado. Variables de entrenamiento eliminadas para liberar memoria.")

--- Tabla Comparativa de Modelos ---
               Model         RMSE  Training Time (s)  Prediction Time (s)
0   LightGBM (Tuned)  1574.842213          22.913460             0.096004
1      Random Forest  1635.804462           7.418568             0.592802
2      Decision Tree  2149.348996           0.768487             0.028800
3  Linear Regression  2577.292228           3.662256             0.082954

--- Conclusión del Proyecto ---
El mejor modelo en términos de calidad de predicción (menor RMSE) es 'LightGBM (Tuned)' con un RMSE de 1574.84.

Análisis general:
1. CALIDAD: Los modelos de boosting como LightGBM y de ensemble como Random Forest suelen superar a los modelos simples como la Regresión Lineal y el Árbol de Decisión.
2. VELOCIDAD: La Regresión Lineal es el modelo más rápido de entrenar, pero su calidad es baja. LightGBM es notablemente rápido tanto en entrenamiento como en predicción, a menudo superando a Random Forest.
3. BALANCE: 'LightGBM (Tuned)' parece ofrecer el mejo

## Conclusiones y Observaciones Finales

Tras el entrenamiento y evaluación de los diferentes modelos de regresión para la predicción de precios de automóviles, se pueden destacar los siguientes puntos clave:

**Resultados de los Modelos:**
- El modelo LightGBM ajustado mediante GridSearchCV obtuvo el menor RMSE, demostrando una excelente capacidad predictiva y tiempos de entrenamiento y predicción muy competitivos.
- Random Forest también mostró un buen desempeño, aunque con tiempos de entrenamiento superiores a LightGBM.
- Los modelos más simples, como la Regresión Lineal y el Árbol de Decisión, ofrecieron mayor velocidad de entrenamiento, pero su precisión fue considerablemente menor.

**Observaciones Generales del Proyecto:**
- La calidad de la predicción es fundamental, pero debe ser balanceada con la eficiencia computacional, especialmente en entornos de producción o con recursos limitados.
- El preprocesamiento de los datos y la correcta selección de características influyeron significativamente en el rendimiento de los modelos.
- La comparación sistemática de los modelos permitió identificar ventajas y desventajas de cada enfoque, facilitando la toma de decisiones informada.
- La eliminación de variables innecesarias y la gestión eficiente de la memoria son prácticas recomendadas en notebooks con recursos limitados.

**Conclusión Final:**
El modelo seleccionado, LightGBM ajustado, representa la mejor opción para este problema específico, ya que combina alta precisión con tiempos de ejecución razonables. Sin embargo, la elección del modelo ideal puede variar según las restricciones y necesidades del proyecto, por lo que siempre es recomendable realizar un análisis integral considerando tanto la calidad como la eficiencia.


# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos